## Hospital General Information Dataset

### By: Anurag Bolneni

This notebook has our preliminary hospital reccomendation system MVP based on the Hospital General Information dataset from CMS. It is split into two sections:
+ Step 1: Data Cleaning & Manipulation
+ Step 2: Taking User Input
+ Step 3: Reccomendation System MVP

## Step 1: Data Cleaning & Manipulation

We first import necessary libraries and use an API to pull data from CMS websites. Then, data is fed into a pandas dataframe where we clean for parameters of interest and data types of different columns.

In [1]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
pd.read_csv('Data/Hospital_General_Information.csv').head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,...,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,READM Group Footnote,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,Pt Exp Group Footnote,TE Group Measure Count,Count of Facility TE Measures,TE Group Footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,...,1,9,1,NaN,8,8,NaN,14,11,NaN
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,...,0,9,1,NaN,8,8,NaN,14,14,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 768-8400,Acute Care Hospitals,Proprietary,...,1,7,1,NaN,8,8,NaN,14,11,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,...,0,6,0,NaN,8,8,NaN,14,7,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,Acute Care Hospitals,Proprietary,...,0,4,0,NaN,8,Not Available,5.0,14,8,NaN


In [3]:
df_hosp_gen_info = pd.read_csv('Data/Hospital_General_Information.csv').iloc[:,:13].drop(columns=['Phone Number','Meets criteria for promoting interoperability of EHRs'])
df_hosp_gen_info.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,Yes,2
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,Acute Care Hospitals,Proprietary,Yes,2
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,Acute Care Hospitals,Voluntary non-profit - Private,Yes,2
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,Acute Care Hospitals,Proprietary,Yes,2


In [4]:
print('Length of unique Facility ID vs total:',len(df_hosp_gen_info['Facility ID'].unique()),',',len(df_hosp_gen_info['Facility ID']))
print('Length of unique address vs total:',len(df_hosp_gen_info.Address.unique()),',',len(df_hosp_gen_info.Address))
print('Variations in hospital overall ratings:', Counter(df_hosp_gen_info['Hospital overall rating']))
print('Emerg_services:', Counter(df_hosp_gen_info['Emergency Services']))
df_hosp_gen_info.dtypes

Length of unique Facility ID vs total: 5306 , 5306
Length of unique address vs total: 5276 , 5306
Variations in hospital overall ratings: Counter({'Not Available': 1996, '3': 1006, '4': 979, '2': 682, '5': 452, '1': 191})
Emerg_services: Counter({'Yes': 4455, 'No': 851})


Facility ID                object
Facility Name              object
Address                    object
City                       object
State                      object
ZIP Code                    int64
County Name                object
Hospital Type              object
Hospital Ownership         object
Emergency Services         object
Hospital overall rating    object
dtype: object

-- Anurag to write blurb about above findings later

We'll be taking the cosine similarity of State and Emergency Services to test our MVP of the reccomendation algorithm. As a last step of cleaning, we need to change Emergency Services (Y/N) to binary classifications.

In [5]:
df_hosp_gen_info['Emergency Services'] = [1 if x=='Yes' else 0 for x in df_hosp_gen_info['Emergency Services']]
df_hosp_gen_info.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,1,3
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,1,2
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,Acute Care Hospitals,Proprietary,1,2
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,Acute Care Hospitals,Voluntary non-profit - Private,1,2
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,Acute Care Hospitals,Proprietary,1,2


In [6]:
df_HCAHPS = pd.read_csv('Data/HCAHPS-Hospital.csv')
df_HCAHPS = df_HCAHPS[['Facility ID','HCAHPS Measure ID','HCAHPS Question','HCAHPS Answer Percent']]
df_HCAHPS['HCAHPS Answer Percent'] = pd.to_numeric(df_HCAHPS['HCAHPS Answer Percent'], errors='coerce')
df_HCAHPS = df_HCAHPS.dropna(axis=0)

df_info = df_HCAHPS.groupby(['Facility ID']).count()
df = df_info[df_info['HCAHPS Question'] == 72].reset_index()
VALID_FACILITY_IDS = list(df['Facility ID'])

df_HCAHPS = df_HCAHPS[df_HCAHPS['Facility ID'].isin(VALID_FACILITY_IDS) == True]

TEST_ = df_HCAHPS[df_HCAHPS['Facility ID'].isin(VALID_FACILITY_IDS) == True]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (12,14,17,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
%%time
# Here's how to calculate summary metrics using group_by and creating some new columns
question_type_dict = {'H_COMP_1_A_P': "nurses", 
                      'H_NURSE_RESPECT_A_P': "nurses", 
                      'H_NURSE_LISTEN_A_P': "nurses", 
                      'H_NURSE_EXPLAIN_A_P': "nurses",
                      'H_COMP_2_A_P': "doctors",
                      'H_DOCTOR_RESPECT_A_P': "doctors",
                      'H_DOCTOR_LISTEN_A_P': "doctors", 
                      'H_DOCTOR_EXPLAIN_A_P': "doctors",
                      'H_COMP_3_A_P': "patients", 
                      'H_CALL_BUTTON_A_P': "patients", 
                      'H_BATH_HELP_A_P': "patients", 
                      'H_COMP_5_A_P': "staffs", 
                      'H_MED_FOR_A_P': "staffs", 
                      'H_SIDE_EFFECTS_A_P': "staffs"
                     }
df_HCAHPS["measurement_type"] = df_HCAHPS.apply(lambda row: question_type_dict[row["HCAHPS Measure ID"]] if row["HCAHPS Measure ID"] in question_type_dict.keys() else "UNKNOWN", axis=1)
grouped = df_HCAHPS.groupby(['Facility ID', 'measurement_type']).mean()
grouped = grouped.drop("UNKNOWN", level="measurement_type").reset_index()

Wall time: 8.48 s


In [8]:
df_HCAHPS

,Facility ID,HCAHPS Measure ID,HCAHPS Question,HCAHPS Answer Percent,measurement_type
0,010001,H_COMP_1_A_P,"Patients who reported that their nurses ""Alway...",77.0,nurses
1,010001,H_COMP_1_SN_P,"Patients who reported that their nurses ""Somet...",7.0,UNKNOWN
2,010001,H_COMP_1_U_P,"Patients who reported that their nurses ""Usual...",16.0,UNKNOWN
5,010001,H_NURSE_RESPECT_A_P,"Patients who reported that their nurses ""Alway...",81.0,nurses
6,010001,H_NURSE_RESPECT_SN_P,"Patients who reported that their nurses ""Somet...",7.0,UNKNOWN
7,010001,H_NURSE_RESPECT_U_P,"Patients who reported that their nurses ""Usual...",12.0,UNKNOWN
8,010001,H_NURSE_LISTEN_A_P,"Patients who reported that their nurses ""Alway...",73.0,nurses
9,010001,H_NURSE_LISTEN_SN_P,"Patients who reported that their nurses ""Somet...",7.0,UNKNOWN
10,010001,H_NURSE_LISTEN_U_P,"Patients who reported that their nurses ""Usual...",20.0,UNKNOWN
11,010001,H_NURSE_EXPLAIN_A_P,"Patients who reported that their nurses ""Alway...",77.0,nurses


In [9]:
d = ['nurses', 'doctors', 'staffs', 'patients']
clinician_df = grouped[['Facility ID']]

for i in range(4):
    CLINICIANS = grouped[grouped['measurement_type']==d[i]][['Facility ID', 'HCAHPS Answer Percent']]
    CLINICIANS.rename(columns = {'HCAHPS Answer Percent':d[i]}, inplace = True)
    CLINICIANS[d[i]] = CLINICIANS[d[i]]
    clinician_df = clinician_df.merge(CLINICIANS, on='Facility ID')
    
clinician_df = clinician_df.drop_duplicates().dropna()


In [10]:
new = df_hosp_gen_info.merge(clinician_df, on='Facility ID').dropna().reset_index()
print('We have '+ str(len(new)) + ' facilties in the df after joining and dropping NA')
new = new.drop(labels=['index','Hospital Type','Hospital Ownership'], axis=1)
new

We have 3531 facilties in the df after joining and dropping NA


,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Emergency Services,Hospital overall rating,nurses,doctors,staffs,patients
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,1,3,77.00,80.75,63.666667,60.666667
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,1,2,75.00,84.00,56.000000,47.666667
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,1,2,69.00,77.00,55.333333,44.666667
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,1,2,84.00,85.75,77.333333,72.666667
4,010011,ST. VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,JEFFERSON,1,2,77.25,79.25,63.000000,65.333333
5,010012,DEKALB REGIONAL MEDICAL CENTER,200 MED CENTER DRIVE,FORT PAYNE,AL,35968,DE KALB,1,3,77.00,78.50,51.000000,60.666667
6,010016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,SHELBY,1,4,76.25,80.00,62.000000,58.333333
7,010019,HELEN KELLER HOSPITAL,1300 SOUTH MONTGOMERY AVENUE,SHEFFIELD,AL,35660,COLBERT,1,3,85.25,87.75,60.333333,65.666667
8,010021,DALE MEDICAL CENTER,126 HOSPITAL AVE,OZARK,AL,36360,DALE,1,4,79.75,79.75,67.333333,64.333333
9,010023,BAPTIST MEDICAL CENTER SOUTH,2105 EAST SOUTH BOULEVARD,MONTGOMERY,AL,36116,MONTGOMERY,1,2,78.75,81.00,66.000000,67.666667


## Step 2: Take User Input

In [11]:
#columns = ['Hospital overall rating','Emergency Services','State']
columns = ['Emergency Services','State']
df = pd.DataFrame(columns = columns, dtype=object)

def user_input(df):
    # Take user input for a series of factors
    return df.append({
        #'Hospital overall rating': int(input('Please rate your desired hospital on scale of 1-5:   ')),
        'Emergency Services': int(input('Do you need emergency services? (Yes = 1, No = 0)     ')),
        'State':str(input('Which State do you live in?     ')),
        'Doctors': int(input('Rate your ideal doctor on a scale of 0-100%     ')),
        'Nurses': int(input('Rate your ideal nurses on a scale of 0-100%     ')),
        'Patients': int(input('Rate your ideal patients on a scale of 0-100%     ')),
        'Staffs': int(input('Rate your ideal staffs on a scale of 0-100%     '))},
    ignore_index = True)

In [12]:
user_input_df = user_input(df)
user_input_df.head()

Do you need emergency services? (Yes = 1, No = 0)     1
Which State do you live in?     MI
Rate your ideal doctor on a scale of 0-100%     86
Rate your ideal nurses on a scale of 0-100%     54
Rate your ideal patients on a scale of 0-100%     39
Rate your ideal staffs on a scale of 0-100%     6


,Emergency Services,State,Doctors,Nurses,Patients,Staffs
0,1,MI,86.0,54.0,39.0,6.0


## Step 3: Reccommendar System MVP

Our hospital reccomendation system utilizes the cleaned dataset from Step 1 and the User Input from Step 2. We use cosine similarity to determine the top hospitals based on the users target inputs. This section is broken into a set of function that to conduct vectorization of string parmaters as well as numeric parameters to determine cosine similarity of hospitals relative to a user's need. 

For our MVP, we tested our recommender sytem on Emergency Services (Y/N) and State of hospital as initial inputs. Finally, we sort the outcomes by cosine to determine the list of best hospitals. The outcomes so far look like it works relatively well, but we'll need to see how this fluctuates with additional parameters and available data.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
def give_me_hospitals(df_hospital, df_patient):
    
    #Collaborative Filtering
    Patient_Preferences = ['State','Emergency Services']
    for values in Patient_Preferences:
        df_hospital = df_hospital[df_hospital[values] == df_patient[values][0]]
        
    #df_hospital = df_hospital[df_hospital['Hospital overall rating'] == str(df_patient['Hospital overall rating'][0])]
    
    # Begin attaining X parameters directly based on hospital DF
    X_no2 = np.array([df_hospital['doctors'],
                      df_hospital['nurses'],
                      df_hospital['patients'],
                      df_hospital['staffs']]).reshape([-1,4], order='F')
    
    # Begin attaining Y paramters directly based on patient DF
    Y_Final = np.array([df_patient['Doctors'],
                        df_patient['Nurses'],
                        df_patient['Patients'],
                        df_patient['Staffs']]).reshape(-1,4)

    Cos = cosine_similarity(X_no2,Y_Final)
    df_hospital['Cosine Similarity'] = [values[0] for values in Cos]

    return df_hospital.sort_values('Cosine Similarity', ascending=False).reset_index()

In [15]:
df_hospital = new.copy()
df_patient = user_input_df.copy()
give_me_hospitals(df_hospital, df_patient)

,index,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Emergency Services,Hospital overall rating,nurses,doctors,staffs,patients,Cosine Similarity
0,1639,230273,DETROIT RECEIVING HOSPITAL,4201 ST ANTOINE ST - 2C,DETROIT,MI,48201,WAYNE,1,1,70.00,69.25,53.000000,53.000000,0.895888
1,1563,230017,BRONSON METHODIST HOSPITAL,601 JOHN STREET,KALAMAZOO,MI,49007,KALAMAZOO,1,4,83.75,83.25,67.000000,71.000000,0.894253
2,1564,230019,"ASCENSION PROVIDENCE HOSPITAL, SOUTHFIELD AND ...",16001 W NINE MILE RD,SOUTHFIELD,MI,48075,OAKLAND,1,4,76.75,75.25,58.000000,63.333333,0.876890
3,1561,230005,PROMEDICA CHARLES AND VIRGINIA HICKMAN HOSPITAL,5640 N ADRIAN HIGHWAY,ADRIAN,MI,49221,LENAWEE,1,5,78.25,81.50,59.666667,68.000000,0.876470
4,1573,230036,MIDMICHIGAN MEDICAL CENTER - ALPENA,1501 W CHISHOLM ST,ALPENA,MI,49707,ALPENA,1,4,83.25,81.00,66.000000,77.333333,0.870224
5,1559,230002,ST JOE MERCY HOSPITAL SYSTEM LIVONIA,36475 FIVE MILE ROAD,LIVONIA,MI,48154,WAYNE,1,3,81.00,76.75,60.666667,68.000000,0.869599
6,1571,230031,LAKE HURON MEDICAL CENTER,2601 ELECTRIC AVENUE,PORT HURON,MI,48060,ST. CLAIR,1,5,76.25,73.75,59.000000,64.666667,0.868663
7,1598,230097,MUNSON MEDICAL CENTER,1105 SIXTH STREET,TRAVERSE CITY,MI,49684,GRAND TRAVERSE,1,5,81.75,80.25,63.000000,65.000000,0.866750
8,1646,231304,HELEN NEWBERRY JOY HOSPITAL,502 W HARRIE ST,NEWBERRY,MI,49868,LUCE,1,Not Available,86.75,88.75,78.333333,80.333333,0.865838
9,1599,230099,PROMEDICA MONROE REGIONAL HOSPITAL,718 N MACOMB ST,MONROE,MI,48162,MONROE,1,3,80.75,77.25,60.000000,60.000000,0.858942


### Evaluation

In [16]:
import random

random.seed(42)

In [17]:
def rand_query_generator(n=5):
    """
    generates n random queries based on six input parameters incorporated in the recommender system
    """
    
    queries = []
    
    for i in range(n):
        query = []
        query.append(random.randint(0,1))
        query.append(random.choice(new.State.unique()))
        query.append(random.randint(70,100))
        query.append(random.randint(50,100))
        query.append(random.randint(0,100))
        query.append(random.randint(0,100))
        
        queries.append(query)
        
    df_queries = pd.DataFrame(queries, columns=['Emergency Services','State', 'Doctors', 'Nurses', 'Patients', 'Staffs'])
    
    return df_queries

In [22]:
queries = rand_query_generator()
queries

,Emergency Services,State,Doctors,Nurses,Patients,Staffs
0,0,OH,79,90,79,46
1,0,UT,72,52,84,29
2,1,CO,97,64,12,48
3,1,NH,90,73,20,47
4,1,IL,91,67,89,87


#### Precision and Recall

In [23]:
def evaluation_pre_rec_at_n(df_hospital, queries, n=10):
    """
    takes overall hospital data and evaluates precision and recall for 
    multiple queries using the relevant and recommended hospitals
    """
    
    pre_at_n, rec_at_n = {}, {}
    
    for i in range(len(queries)):
        print(i)
        try:
            # obtaining hospital recommendations
            df_patient = queries.iloc[i:i+1].reset_index(drop=True)
            #df_hospital = df_hospital.drop('Hospital overall rating', axis=1)

            hosp_rec = give_me_hospitals(df_hospital, df_patient)
            
            print('query #' + str(i+1) + ': ' + str(len(hosp_rec)) + ' recommendations generated')

            hosp_rel = df_hospital[df_hospital['State'] == df_patient['State'][0]]
            hosp_rel.replace('Not Available', np.nan).sort_values(by=['Hospital overall rating', 'Emergency Services'], 
                                                                  ascending=False)
            hosp_rel = hosp_rel[:10]
            
            
            # calculating precision and recall for each query
            if n == -1 or n > len(hosp_rec):
                rel_rec = list(set(hosp_rec['Facility ID']).intersection(hosp_rel['Facility ID']))
                pre_at_n['q'+str(i+1)] = len(rel_rec) / len(hosp_rec)
                rec_at_n['q'+str(i+1)] = len(rel_rec) / len(hosp_rel)

            else:
                rel_rec = list(set(hosp_rec['Facility ID'][:n]).intersection(hosp_rel['Facility ID']))
                pre_at_n['q'+str(i+1)] = len(rel_rec) / len(hosp_rec[:n])
                rec_at_n['q'+str(i+1)] = len(rel_rec) / len(hosp_rel)
        
        except ValueError:
            print('query #' + str(i+1) + ': ' + 'Too few or no hospitals found in the database.')
            pre_at_n['q'+str(i+1)] = np.nan
            rec_at_n['q'+str(i+1)] = np.nan
        
        except KeyError:
            print('query #' + str(i+1) + ': ' + 'Too few or no hospitals found in the database.')
            pre_at_n['q'+str(i+1)] = np.nan
            rec_at_n['q'+str(i+1)] = np.nan
    
    return pre_at_n, rec_at_n

In [24]:
evaluation_pre_rec_at_n(new, queries)

0
query #1: 18 recommendations generated
1
query #2: Too few or no hospitals found in the database.
2
query #3: 65 recommendations generated
3
query #4: 22 recommendations generated
4
query #5: 138 recommendations generated


({'q1': 0.1, 'q2': nan, 'q3': 0.2, 'q4': 0.7, 'q5': 0.0},
 {'q1': 0.1, 'q2': nan, 'q3': 0.2, 'q4': 0.7, 'q5': 0.0})

#### Mean Average Precision

In [25]:
def evaluation_avg_pre(df_hospital, queries, cutoff=10):
    """
    takes overall hospital data and evaluates (mean) average precision for 
    multiple queries using the relevant and recommended hospitals
    """
    
    avg_pre, mean_avg_pre = {}, {}
    
    for i in range(len(queries)):
        
        rel_counter =  0
        pre = []
        
        try:
            # obtaining hospital recommendations
            df_patient = queries.iloc[i:i+1].reset_index(drop=True)
            
            hosp_rec = give_me_hospitals(df_hospital, df_patient)
            
            print('query #' + str(i+1) + ': ' + str(len(hosp_rec)) + ' recommendations generated')

            hosp_rel = df_hospital[df_hospital['State'] == df_patient['State'][0]]
            hosp_rel.replace('Not Available', np.nan).sort_values(by=['Hospital overall rating', 'Emergency Services'], 
                                                                  ascending=False)
            hosp_rel = hosp_rel[:10]
            
            
            # calculating average precision for each query
            if cutoff == -1 or cutoff > len(hosp_rec):
                for j, hosp in enumerate(hosp_rec['Facility ID']):
                    if hosp in hosp_rel['Facility ID']:
                        rel_counter += 1
                        pre.append(rel_counter/(j+1))
            else:
                for j, hosp in enumerate(hosp_rec['Facility ID'][:cutoff]):
                    if hosp in hosp_rel['Facility ID']:
                        rel_counter += 1
                        pre.append(rel_counter/(j+1))

            avg_pre['q'+str(i+1)] = sum(pre) / len(hosp_rel)
            
        except ValueError:
            print('query #' + str(i+1) + ': ' + 'Too few or no hospitals found in the database.')
        
        except KeyError:
            print('query #' + str(i+1) + ': ' + 'Too few or no hospitals found in the database.')
    
    
    # calculating mean of average precisions for all queries
    mean_avg_pre = sum(avg_pre.values()) / len(avg_pre.values())
    
    return avg_pre, mean_avg_pre

In [26]:
evaluation_avg_pre(new, queries)

query #1: 18 recommendations generated
query #2: Too few or no hospitals found in the database.
query #3: 65 recommendations generated
query #4: 22 recommendations generated
query #5: 138 recommendations generated


({'q1': 0.0, 'q3': 0.0, 'q4': 0.0, 'q5': 0.0}, 0.0)

#### Normalized DCG

In [27]:
import math

def evaluation_nDCG_at_n(df_hospital, queries, n=10, base=2):
    """
    takes overall hospital data and evaluates normalized DCG for 
    multiple queries using the relevant and recommended hospitals
    """
    
    ndcg = {}
    
    for i in range(len(queries)):
        
        try:
            # obtaining hospital recommendations
            df_patient = queries.iloc[i:i+1].reset_index(drop=True)
            
            hosp_rec = give_me_hospitals(df_hospital, df_patient)
            
            print('query #' + str(i+1) + ': ' + str(len(hosp_rec)) + ' recommendations generated')

            hosp_rel = df_hospital[df_hospital['State'] == df_patient['State'][0]]
            hosp_rel.replace('Not Available', np.nan).sort_values(by=['Hospital overall rating', 'Emergency Services'], 
                                                                  ascending=False)
            hosp_rel = hosp_rel[:10]
            
            
            # calculating relevant hospital ranks
            rank = len(hosp_rel) + 1
            rel = {}
            
            for h in hosp_rel['Facility ID']:
                rel[h] = rank
                rank -= 1
            
            
            # calculating system order ranks for recommended hospitals
            rec_sys = {}

            for h in hosp_rec['Facility ID']:
                if h in hosp_rel['Facility ID']:
                    rec_sys[h] = rel[h]
                else:
                    rec_sys[h] = 1
            
            
            # calculating dcg and ndcg
            if n == -1 or n > len(hosp_rec):
                sys_scores = list(rec_sys.values())
                idl_scores = list(rel.values())
            else:
                sys_scores = list(rec_sys.values())[:n]
                idl_scores = list(rel.values())[:n]

            if len(idl_scores) < len(sys_scores):
                idl_scores += [1] * (len(sys_scores)-len(idl_scores))
            
            sys_list = []
            idl_list = []

            for j, (sys, idl) in enumerate(zip(sys_scores, idl_scores)):

                sys_rate = sys / math.log(j+1, base) if j > (base-2) else sys
                sys_list.append(sys_rate)

                idl_rate = idl / math.log(j+1, base) if j > (base-2) else idl
                idl_list.append(idl_rate) 

            sys_dcg = sum(sys_list)
            idl_dcg = sum(idl_list)

            ndcg['q'+str(i+1)] = sys_dcg / idl_dcg
            
            
        except ValueError:
            print('query #' + str(i+1) + ': ' + 'Too few or no hospitals found in the database.')
            
        except KeyError:
            print('query #' + str(i+1) + ': ' + 'Too few or no hospitals found in the database.')
        
    return ndcg

In [28]:
evaluation_nDCG_at_n(new, queries)

query #1: 18 recommendations generated
query #2: Too few or no hospitals found in the database.
query #3: 65 recommendations generated
query #4: 22 recommendations generated
query #5: 138 recommendations generated


{'q1': 0.12917591694169767,
 'q3': 0.12917591694169767,
 'q4': 0.12917591694169767,
 'q5': 0.12917591694169767}